In [0]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs

In [0]:
!ls german-traffic-signs

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import random
import pickle
import pandas as pd
import cv2

In [0]:
np.random.seed(0)

In [0]:
with open('german-traffic-signs/train.p', 'rb') as f:
    train_data = pickle.load(f)
with open('german-traffic-signs/valid.p', 'rb') as f:
    val_data = pickle.load(f)
with open('german-traffic-signs/test.p', 'rb') as f:
    test_data = pickle.load(f)

print(type(train_data))

xtrain_data, ytrain_data = train_data['features'], train_data['labels']
xval_data, yval_data = val_data['features'], val_data['labels']
xtest_data, ytest_data = test_data['features'], test_data['labels']


In [0]:
print(xtrain_data.shape)
print(xval_data.shape)
print(xtest_data.shape)

In [0]:
assert(xtrain_data.shape[0] == ytrain_data.shape[0]), "The number of images is not equal to number of labels."
assert(xtest_data.shape[0] == ytest_data.shape[0]), "The number of images is not equal to number of labels."
assert(xval_data.shape[0] == yval_data.shape[0]), "The number of images is not equal to number of labels."
assert(xtrain_data.shape[1:] == (32,32,3)), "The dimentions of image is not equal to 32x32"
assert(xtest_data.shape[1:] == (32,32,3)), "The dimentions of image is not equal to 32x32"
assert(xval_data.shape[1:] == (32,32,3)), "The dimentions of image is not equal to 32x32"

In [0]:
data = pd.read_csv('german-traffic-signs/signnames.csv')
num_of_samples = []

cols = 5
num_classes = 43

fig, axs = plt.subplots(nrows=num_classes, ncols = cols, figsize=(5, 50))
fig.tight_layout()
for i in range(cols):
    for j, row in data.iterrows():
        x_selected = xtrain_data[ytrain_data == j]
        axs[j][i].imshow(x_selected[random.randint(0, len(x_selected - 1)), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j) + "-" + row['SignName'])
            num_of_samples.append(len(x_selected))

In [0]:
print(num_of_samples)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")

In [0]:
def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img

In [0]:
xtrain_data = np.array(list(map(preprocess_image, xtrain_data)))
xval_data = np.array(list(map(preprocess_image, xval_data)))
xtest_data = np.array(list(map(preprocess_image, xtest_data)))

In [0]:
xtrain_data = xtrain_data.reshape(xtrain_data.shape[0], 32, 32, 1)
xval_data = xval_data.reshape(xval_data.shape[0], 32, 32, 1)
xtest_data = xtest_data.reshape(xtest_data.shape[0], 32, 32, 1)

In [0]:
datagen = ImageDataGenerator(
                width_shift_range=0.1,
                height_shift_range=0.1,
                zoom_range=0.2,
                shear_range=0.1,
                rotation_range=10)

datagen.fit(xtrain_data)

In [0]:
batches = datagen.flow(xtrain_data, ytrain_data, batch_size=20)
xbatch_data, ybatch_data = next(batches)

fig1, axs1 = plt.subplots(1, 15, figsize=(20, 5))
fig1.tight_layout()
for i in range(15):
    axs1[i].imshow(xbatch_data[i].reshape(32,32), cmap=plt.get_cmap("gray"))
    axs1[i].axis("off")

In [0]:
print(xtrain_data.shape)
print(xval_data.shape)
print(xtest_data.shape)

In [0]:
ytrain_data = to_categorical(ytrain_data, 43) # one hot encoding
yval_data = to_categorical(yval_data, 43) # one hot encoding
ytest_data = to_categorical(ytest_data, 43) # one hot encoding

In [0]:
def lenet_model():
    model = Sequential()
    model.add(Conv2D(60, (5,5), input_shape=(32,32,1), activation="relu"))
    model.add(Conv2D(60, (5,5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(30, (3,3), activation="relu"))
    model.add(Conv2D(30, (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(500, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
model = lenet_model()
print(model.summary())

In [0]:
#history = model.fit(xtrain_data, ytrain_data, epochs=10, validation_data=(xval_data, yval_data), 
#verbose=1, batch_size=400, shuffle=1)
history = model.fit_generator(datagen.flow(xtrain_data, ytrain_data, batch_size=50),
                              steps_per_epoch=2000, epochs=10, validation_data=(xval_data, yval_data), 
                              shuffle=1)

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['accuracy', 'val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')

In [0]:
score = model.evaluate(xtest_data, ytest_data)
print(score)
print("Test score", score[0])
print("Test accuracy", score[1])

In [0]:
#fetch image
  
import requests
from PIL import Image
#https://c8.alamy.com/comp/G667W0/road-sign-speed-limit-30-kmh-zone-passau-bavaria-germany-G667W0.jpg

#https://c8.alamy.com/comp/A0RX23/cars-and-automobiles-must-turn-left-ahead-sign-A0RX23.jpg

#https://previews.123rf.com/images/bwylezich/bwylezich1608/bwylezich160800375/64914157-german-road-sign-slippery-road.jpg

#https://previews.123rf.com/images/pejo/pejo0907/pejo090700003/5155701-german-traffic-sign-no-205-give-way.jpg

#https://c8.alamy.com/comp/J2MRAJ/german-road-sign-bicycles-crossing-J2MRAJ.jpg


url = 'https://c8.alamy.com/comp/A0RX23/cars-and-automobiles-must-turn-left-ahead-sign-A0RX23.jpg'
r = requests.get(url, stream=True)
img = Image.open(r.raw)
plt.imshow(img, cmap=plt.get_cmap('gray'))


In [0]:
#Preprocess image
  
img = np.asarray(img)
img = cv2.resize(img, (32, 32))
img = preprocess_image(img)
plt.imshow(img, cmap = plt.get_cmap('gray'))
print(img.shape)
  
#Reshape reshape
  
img = img.reshape(1, 32, 32, 1)
  
#Test image
print("predicted sign: "+ str(model.predict_classes(img)))